In [1]:
import numpy as np 
import tensorflow as tf
import wave, math, struct 

from keras.models import Sequential
from keras.layers import Dense,LSTM,Activation
from keras.utils import to_categorical

In [2]:
# prepare a dummy data for simulating notes
notes_freqs = {
    'A' : 440.0,'B' : 493.88,'C' : 261.63, 'D' : 293.66,
    'E' : 393.63, 'F' : 349.23, 'G' : 392.00
}

In [3]:
notes_freqs

{'A': 440.0,
 'B': 493.88,
 'C': 261.63,
 'D': 293.66,
 'E': 393.63,
 'F': 349.23,
 'G': 392.0}

In [4]:
notes = notes_freqs.keys()

In [5]:
notes = list(notes)

In [6]:
note_to_int = {note : i for i,note in enumerate(notes)}
note_to_int

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}

In [7]:
int_to_note = {i : note for i,note in enumerate(notes)}
int_to_note

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G'}

In [8]:
raw_music_data = [notes[np.random.randint(0,7)] for i in range(1000)]
raw_music_data

['F',
 'G',
 'C',
 'D',
 'E',
 'A',
 'G',
 'B',
 'A',
 'C',
 'C',
 'E',
 'G',
 'A',
 'D',
 'F',
 'C',
 'A',
 'B',
 'F',
 'A',
 'A',
 'C',
 'B',
 'G',
 'C',
 'D',
 'D',
 'E',
 'B',
 'E',
 'C',
 'C',
 'B',
 'B',
 'B',
 'E',
 'C',
 'D',
 'C',
 'A',
 'G',
 'A',
 'G',
 'F',
 'G',
 'C',
 'D',
 'D',
 'B',
 'D',
 'G',
 'A',
 'D',
 'D',
 'D',
 'D',
 'E',
 'C',
 'G',
 'C',
 'A',
 'D',
 'D',
 'B',
 'E',
 'F',
 'F',
 'G',
 'G',
 'C',
 'F',
 'G',
 'A',
 'G',
 'D',
 'E',
 'F',
 'A',
 'A',
 'F',
 'G',
 'C',
 'A',
 'B',
 'E',
 'A',
 'E',
 'G',
 'F',
 'B',
 'D',
 'D',
 'F',
 'B',
 'A',
 'C',
 'F',
 'A',
 'E',
 'B',
 'B',
 'E',
 'F',
 'A',
 'E',
 'A',
 'A',
 'A',
 'C',
 'F',
 'E',
 'G',
 'E',
 'G',
 'D',
 'F',
 'C',
 'C',
 'A',
 'G',
 'C',
 'C',
 'B',
 'G',
 'B',
 'A',
 'A',
 'D',
 'C',
 'A',
 'B',
 'D',
 'C',
 'A',
 'E',
 'D',
 'A',
 'F',
 'G',
 'B',
 'D',
 'D',
 'E',
 'G',
 'D',
 'F',
 'E',
 'C',
 'F',
 'A',
 'G',
 'C',
 'B',
 'A',
 'E',
 'B',
 'B',
 'E',
 'G',
 'A',
 'A',
 'G',
 'D',
 'F',
 'B',
 'D'

<h1>Data Preparation</h1>

In [9]:
sequence_lenght = 3
network_input = []
network_output = []

for i in range(len(raw_music_data) - sequence_lenght):
    seq_in = raw_music_data[i:i+sequence_lenght]
    seq_out = raw_music_data[i+sequence_lenght]
    network_input.append([note_to_int[char] for char in seq_in])
    network_output.append(note_to_int[seq_out])
    print(seq_in,'-->',seq_out)
    

['F', 'G', 'C'] --> D
['G', 'C', 'D'] --> E
['C', 'D', 'E'] --> A
['D', 'E', 'A'] --> G
['E', 'A', 'G'] --> B
['A', 'G', 'B'] --> A
['G', 'B', 'A'] --> C
['B', 'A', 'C'] --> C
['A', 'C', 'C'] --> E
['C', 'C', 'E'] --> G
['C', 'E', 'G'] --> A
['E', 'G', 'A'] --> D
['G', 'A', 'D'] --> F
['A', 'D', 'F'] --> C
['D', 'F', 'C'] --> A
['F', 'C', 'A'] --> B
['C', 'A', 'B'] --> F
['A', 'B', 'F'] --> A
['B', 'F', 'A'] --> A
['F', 'A', 'A'] --> C
['A', 'A', 'C'] --> B
['A', 'C', 'B'] --> G
['C', 'B', 'G'] --> C
['B', 'G', 'C'] --> D
['G', 'C', 'D'] --> D
['C', 'D', 'D'] --> E
['D', 'D', 'E'] --> B
['D', 'E', 'B'] --> E
['E', 'B', 'E'] --> C
['B', 'E', 'C'] --> C
['E', 'C', 'C'] --> B
['C', 'C', 'B'] --> B
['C', 'B', 'B'] --> B
['B', 'B', 'B'] --> E
['B', 'B', 'E'] --> C
['B', 'E', 'C'] --> D
['E', 'C', 'D'] --> C
['C', 'D', 'C'] --> A
['D', 'C', 'A'] --> G
['C', 'A', 'G'] --> A
['A', 'G', 'A'] --> G
['G', 'A', 'G'] --> F
['A', 'G', 'F'] --> G
['G', 'F', 'G'] --> C
['F', 'G', 'C'] --> D
['G', 'C',

In [10]:
n_patterns = len(network_input)
n_patterns

997

In [11]:
x = np.reshape(network_input, (n_patterns,sequence_lenght,1))
x

array([[[5],
        [6],
        [2]],

       [[6],
        [2],
        [3]],

       [[2],
        [3],
        [4]],

       ...,

       [[3],
        [6],
        [3]],

       [[6],
        [3],
        [5]],

       [[3],
        [5],
        [2]]], shape=(997, 3, 1))

In [12]:
y = to_categorical(network_output)
y.shape

(997, 7)

In [13]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], shape=(997, 7))

<h1>Build Model</h1>

In [14]:
model = Sequential()
model.add(LSTM(256,input_shape = (x.shape[1],x.shape[2])))
model.add(Dense(1000,activation = 'relu'))
model.add(Dense(7,activation='softmax'))


C:\Users\dai\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics = ['accuracy'])

In [16]:
model.fit(x,y,epochs = 100)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.1294 - loss: 1.9565
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1555 - loss: 1.9464
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1555 - loss: 1.9462
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1655 - loss: 1.9435
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1645 - loss: 1.9425
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1765 - loss: 1.9405
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1565 - loss: 1.9423
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1735 - loss: 1.9412
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1775 - loss: 1.9399
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1695 - loss: 1.9349
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1825 - loss: 1.9344
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy

<h1>Generate a new melody sequence</h1>

In [17]:
start_index = np.random.randint(0,len(network_input))
pattern = network_input[start_index]
pattern

[2, 6, 6]

In [18]:
generated_melody = []

for i in range(16):
    x_input = np.reshape(pattern,(1,len(pattern),1))
    prediction = model.predict(x_input,verbose= 0 )
    index = np.argmax(prediction)
    result = int_to_note[index]
    generated_melody.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
generated_melody

['A',
 'D',
 'G',
 'B',
 'F',
 'G',
 'A',
 'A',
 'C',
 'F',
 'A',
 'E',
 'G',
 'B',
 'D',
 'G']

In [19]:
pattern

[np.int64(1), np.int64(3), np.int64(6)]

<h1>Save this as audio file</h1>

In [20]:
with wave.open('my_music.wav','w') as wav_file:
    wav_file.setparams((1,2,44100,0,'NONE','not compressed'))
    for note in generated_melody:
        freq = notes_freqs[note]
        num_samples = int(0.5 * 44100)
        for i in range(num_samples):;-
            t = float(i)/44100
            value = int(32767 * 0.5 * math.sin(2*math.pi*freq*t))
            data = struct.pack('<h',value)
            wav_file.writeframes(data)

In [21]:
help(wave.open)

Help on function open in module wave:

open(f, mode=None)

